In [1]:
fulldir="/Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/"
name="ppl13/"

In [2]:
import shutil as sh
import cv2
import os
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm
import os.path

In [3]:
# file = open(fulldir+"pixels.txt","a+")

In [4]:
image = fulldir+name
ext = ".bmp"
key = "abcdefghijklm"
i=0

In [5]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0], image_size[1],color

In [6]:
def LogisticEncryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]
    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    L = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1
    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    C = round((L*L_y*10**4) % 256)
    C_r = round((L*L_y*10**4) % 256)
    C_g = round((L*L_y*10**4) % 256)
    C_b = round((L*L_y*10**4) % 256)
    x = 4*(S_x)*(1-S_x)
    y = 4*(S_y)*(1-S_y)
    imageMatrix,dw, dl, color = getImageMatrix(imageName)
    LogisticEncryptionIm = []
    for i in range(dw):
        row = []
        for j in range(dl):
            max_iterations = 10
            iterations_x = 0
            iterations_y = 0
            while x <0.8 and x > 0.2 and iterations_x<=max_iterations:
                x = 4*x*(1-x)
                iterations_x+=1
            #print(iterations_x)
            while y <0.8 and y > 0.2 and iterations_y<=max_iterations:
                y = 4*y*(1-y)
                iterations_y+=1
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                C_r =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][0]) % N) ^ ((C_r + key_list[7]) % N)
                C_g =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][1]) % N) ^ ((C_g + key_list[7]) % N)
                C_b =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][2]) % N) ^ ((C_b + key_list[7]) % N)
                row.append((C_r,C_g,C_b))
                C = C_r
            else:
                C = ((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j]) % N) ^ ((C + key_list[7]) % N)
                row.append(C)
            x = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            y = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        LogisticEncryptionIm.append(row)
    im = Image.new('L', (dw, dl))
    if color:
        im = Image.new('RGB', (dw, dl))
    else: 
        im = Image.new('L', (dw, dl)) # L is for Black and white pixels    
    pix = im.load()
    for x in range(dw):
        for y in range(dl):
            pix[x, y] = LogisticEncryptionIm[x][y]
    return im
    #im.save(imageName[0:-8]+"encryp"+ext, "BMP")

In [7]:
vals=[]

In [8]:
def EncryptionResult(img):
    cnt=0
    face = []
    modelFile = fulldir+"res10_300x300_ssd_iter_140000.caffemodel"
    configFile = fulldir+"deploy.prototxt.txt"
    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (800, 800)), 1.0,(800, 800), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces = net.forward()
    #to draw faces on image
    for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
#           print(x,y,x1,y1)
            if y1>=h:
                y1=h-1
            if x1>=w:
                x1=w-1
            face.append((x, y, x1, y1))
            cv2.rectangle(img, (x, y), (x1-1, y1-1), (255, 0, 0), 1)
    count=0
    for (x, y, x1, y1) in face:
        cropped_image = img[y+1:y1-1, x+1:x1-1]
        filename=fulldir+"new/"+str(count)+ext
        cv2.imwrite(filename, cropped_image)
        imgt = LogisticEncryption(filename, key)
        LogisticResult=cv2.cvtColor(np.array(imgt),cv2.COLOR_RGB2BGR)
        img[y+1:y1-1, x+1:x1-1] = LogisticResult
        cv2.imwrite(fulldir+name+'/res.png', LogisticResult)
        count=count+1
    return img

In [9]:
# Read the video from specified path
cam = cv2.VideoCapture(fulldir+name+name[:-1]+".mp4")
fps = cam.get(cv2.CAP_PROP_FPS)
print(fps)

30.0


In [10]:
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))

In [11]:
currentframe = 0
video_name = 'encryptedvideo.avi'
os.chdir(fulldir+name+"results")
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))

while(True):
    ret,Img = cam.read()
    if ret:
        print("frame:",currentframe)
        frameout=EncryptionResult(Img)
        if currentframe<10:
            cv2.imwrite(fulldir+name+"frames/00"+str(currentframe)+".png", Img)
            cv2.imwrite(fulldir+name+"encrypted_frames/EncryptedResult00"+str(currentframe)+".bmp", frameout)
        elif currentframe<100:
            cv2.imwrite(fulldir+name+"frames/0"+str(currentframe)+".png", Img)
            cv2.imwrite(fulldir+name+"encrypted_frames/EncryptedResult0"+str(currentframe)+".bmp", frameout)
        else:
            cv2.imwrite(fulldir+name+"frames/"+str(currentframe)+".png", Img)
            cv2.imwrite(fulldir+name+"encrypted_frames/EncryptedResult"+str(currentframe)+".bmp", frameout)
        video.write(frameout)
        currentframe += 1
    else:
        break

frame: 0
frame: 1
frame: 2
frame: 3
frame: 4
frame: 5
frame: 6
frame: 7
frame: 8
frame: 9
frame: 10
frame: 11
frame: 12
frame: 13
frame: 14
frame: 15
frame: 16
frame: 17
frame: 18
frame: 19
frame: 20
frame: 21
frame: 22
frame: 23
frame: 24
frame: 25
frame: 26
frame: 27
frame: 28
frame: 29
frame: 30
frame: 31
frame: 32
frame: 33
frame: 34
frame: 35
frame: 36
frame: 37
frame: 38
frame: 39
frame: 40
frame: 41
frame: 42
frame: 43
frame: 44
frame: 45
frame: 46
frame: 47
frame: 48
frame: 49
frame: 50
frame: 51
frame: 52
frame: 53
frame: 54
frame: 55
frame: 56
frame: 57
frame: 58
frame: 59
frame: 60
frame: 61
frame: 62
frame: 63
frame: 64
frame: 65
frame: 66
frame: 67
frame: 68
frame: 69
frame: 70
frame: 71
frame: 72
frame: 73
frame: 74
frame: 75
frame: 76
frame: 77
frame: 78
frame: 79
frame: 80
frame: 81
frame: 82
frame: 83
frame: 84
frame: 85
frame: 86
frame: 87
frame: 88
frame: 89
frame: 90
frame: 91
frame: 92
frame: 93
frame: 94
frame: 95
frame: 96
frame: 97
frame: 98
frame: 99
frame: 100

In [12]:
cv2.destroyAllWindows()
video.release()

In [13]:
enc=cv2.imread("Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage4/ppl13/encrypted_frames/EncryptedResult000.bmp")
orig=cv2.imread("file:///Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage4/ppl13/encrypted_frames/000.png")

np.array_equal(enc,orig)

[ WARN:0@527.087] global loadsave.cpp:244 findDecoder imread_('Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage4/ppl13/encrypted_frames/EncryptedResult000.bmp'): can't open/read file: check file path/integrity
[ WARN:0@527.088] global loadsave.cpp:244 findDecoder imread_('file:///Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage4/ppl13/encrypted_frames/000.png'): can't open/read file: check file path/integrity


True